In [93]:
# | default_exp scraper.Scraper

In [94]:
# | exporti
from __future__ import annotations

import os

from dataclasses import dataclass
from typing import List, Callable, Any
from bs4 import BeautifulSoup

import re
from urllib.parse import urljoin, urlparse
from concurrent.futures import ThreadPoolExecutor

from gdoc_sync.utils import upsert_folder, convert_str_file_name
import gdoc_sync.scraper.driver as dg

from nbdev.showdoc import patch_to, show_doc

In [95]:
# | hide
from selenium.webdriver.common.by import By
from pprint import pprint

In [96]:
# from selenium.webdriver.common.by import By
# import gdoc_sync.scraper.driver as dg

test_urls = [
    "https://domo-support.domo.com/s/article/36004740075",
    "https://domo-support.domo.com/s/topic/0TO5w000000ZlOmGAK/20202023",  # list of articles
    "https://domo-support.domo.com/s/topic/0TO5w000000Zan7GAC/archived-feature-release-notes",  # list of topics
    "https://domo-support.domo.com/s/knowledge-base",
]

drivergenerator = dg.DriverGenerator(debug_prn=True)
driver = drivergenerator.get_webdriver()

test_soup = dg.get_pagesource(
    driver=driver,
    url=test_urls[3],
    search_criteria_tuple=(By.CLASS_NAME, "topic-nav-container"),
    max_sleep_time=15,
    return_soup=True,
)

# test_soup = dg.get_pagesource(
#     driver=driver,
#     url=test_urls[2],
#     search_criteria_tuple=(
#         By.CSS_SELECTOR,
#         f".{', .'.join(['section-list-item', 'article-list-item'] )}",
#     ),
#     max_sleep_time=15,
#     return_soup=True,
# )

str(test_soup)[0:100]

ChromeDriver 120.0.6099.109 (3419140ab665596f21b385ce136419fde0924272-refs/branch-heads/6099@{#1483})



'<html dir="ltr" lang="en-US"><head><title>Knowledge Base</title><meta content="default-src \'self\'; s'

# Utils for Processing Pages

In [97]:
# | export
def remove_query_params_from_url(url):
    u = urlparse(url)
    return urljoin(url, urlparse(url).path)

In [98]:
[remove_query_params_from_url(url) for url in test_urls]

['https://domo-support.domo.com/s/article/36004740075',
 'https://domo-support.domo.com/s/topic/0TO5w000000ZlOmGAK/20202023',
 'https://domo-support.domo.com/s/topic/0TO5w000000Zan7GAC/archived-feature-release-notes',
 'https://domo-support.domo.com/s/knowledge-base']

In [99]:
# | export
def extract_links(
    soup: BeautifulSoup,
    base_url: str = None,
    custom_link_extractor_fn: Callable = None,  # can add custom function for handling URLs
    debug_prn: bool = False,
) -> [str]:
    """returns a list of urls"""

    links_ls = []

    for link in soup.findAll("a"):
        if not link.has_attr("href"):
            continue

        url = link["href"]

        if debug_prn:
            print(url)

        if url.startswith("/") and base_url:
            url = urljoin(base_url, url)

        if base_url and not url.startswith(base_url):
            continue

        if custom_link_extractor_fn:
            url = custom_link_extractor_fn(url, base_url)

        if url in links_ls or not url:
            continue

        links_ls.append(url)

    return list(set(links_ls))

In [100]:
extract_links(
    test_soup,
    base_url="https://domo-support.domo.com",
    debug_prn=False,
)

['https://domo-support.domo.com/s/knowledge-base?language=en_US',
 'https://domo-support.domo.com/s/topic/0TO5w000000ZamwGAC/release-notes?language=en_US']

In [101]:
# | export
def generate_filename_from_url(url, download_folder=None, file_name=None) -> str:
    parsed_url = urlparse(url)

    file_path = "_".join([str for str in parsed_url[2].split("/") if str])

    if download_folder:
        file_path = os.path.join(download_folder, file_path)

    if file_name:
        file_path = os.path.join(file_path, file_name)

    return file_path

In [102]:
# from pprint import pprint

pprint([generate_filename_from_url(url) for url in test_urls])

print("")
print("-- alternative with base_url and different file name")
print("")

pprint([generate_filename_from_url(url, "/SCRAPE", "index.html") for url in test_urls])

['s_article_36004740075',
 's_topic_0TO5w000000ZlOmGAK_20202023',
 's_topic_0TO5w000000Zan7GAC_archived-feature-release-notes',
 's_knowledge-base']

-- alternative with base_url and different file name

['/SCRAPE/s_article_36004740075/index.html',
 '/SCRAPE/s_topic_0TO5w000000ZlOmGAK_20202023/index.html',
 '/SCRAPE/s_topic_0TO5w000000Zan7GAC_archived-feature-release-notes/index.html',
 '/SCRAPE/s_knowledge-base/index.html']


# Scrape_Config

In [103]:
# | export


@dataclass
class Scrape_Config:
    """class for collating data about how to scrape a page
    pass a list of scrape_config into ScrapeFactory
    """

    pattern: re.Pattern  # pattern for matching URLs to appropriate config
    content_extractor_fn: Callable = None  # function for the subset of HTML to extract
    link_extractor_fn: Callable = (
        extract_links  # function for extracting links from soup
    )

    search_element_type: Any = None  # from selenium.webdriver.common.by import By
    search_element_text: str = None

    def get_search_tuple(self) -> Any:
        """used by dg.get_pagesource() function to wait if rendered page has rendered"""

        if not self.search_element_text and self.search_element_type:
            return None

        return (self.search_element_type, self.search_element_text)

    def is_text_match_pattern(self, text, debug_prn: bool = False):
        pattern = re.compile(self.pattern)

        match_pattern = pattern.match(text)

        if debug_prn:
            print({"text": text, "pattern": self.pattern})

        if not match_pattern:
            return False

        if debug_prn:
            print(match_pattern)

        return True

    def __id__(self, other):
        return self.pattern == other.pattern

In [104]:
show_doc(Scrape_Config)

---

[source](https://github.com/jaewilson07/gdoc_sync/blob/main/gdoc_sync/scraper/Scraper.py#L84){target="_blank" style="float:right; font-size:smaller"}

### Scrape_Config

>      Scrape_Config (pattern:re.Pattern, content_extractor_fn:Callable=None,
>                     link_extractor_fn:Callable=<function extract_links>,
>                     search_element_type:Any=None,
>                     search_element_text:str=None)

class for collating data about how to scrape a page
pass a list of scrape_config into ScrapeFactory

In [105]:
# | export


class ScrapeTask_NoDriverProvided(Exception):
    def __init__(self):
        super().__init__("Driver not provided")


class ScrapeTask_NoBaseUrl(Exception):
    def __init__(self):
        super().__init__(
            "no base_url provided to scrape_task. must run as is_test = True"
        )


class ScrapeTask_NoScrapeCrawler(Exception):
    def __init__(self):
        super().__init__(
            "no scrape_crawler provided.  must run scrape_page as is_test = True"
        )


@dataclass
class Scrape_Task:
    """scrape_factory matches a URL to a scrape_config and returns a scrape_task"""

    url: str  # will be assigned after retrieved from scrape_factory

    base_url: str = None
    download_folder: str = None

    generate_filename_fn: Callable = generate_filename_from_url

    max_sleep_time: int = 10
    scrape_config: Scrape_Config = None
    scrape_crawler: Scrape_Crawler = None  # will be optional parent threadpool manager.

    @classmethod
    def _from_factory(
        cls,
        url,
        base_url,
        scrape_config,
        scrape_crawler=None,
        download_folder=None,
    ):
        return cls(
            url=url,
            base_url=base_url,
            download_folder=download_folder,
            scrape_config=scrape_config,
            scrape_crawler=scrape_crawler,
        )

    def _get_pagesource(self, driver=None, debug_prn: bool = False):
        """
        handles fetching the pagesource, html content of a URL
        """

        url = self.url

        driver = (
            driver
            or (
                self.scrape_crawler
                and self.scrape_crawler.driver_generator.get_driver()
            )
            or None
        )

        if not driver:
            raise ScrapeTask_NoDriverProvided()

        search_criteria_tuple = (
            self.scrape_config and self.scrape_config.get_search_tuple()
        ) or None

        max_sleep_time = self.max_sleep_time or 15

        if debug_prn:
            print(
                {
                    "url": url,
                    "max_sleep_time": max_sleep_time,
                    "search_criteria": search_criteria_tuple,
                }
            )

        return dg.get_pagesource(
            url=url,
            search_criteria_tuple=search_criteria_tuple,
            driver=driver,
            max_sleep_time=max_sleep_time,
        )

    def _download_content(self, file_name, content):
        dir_name = os.path.dirname(file_name)

        if dir_name[-1] != "/":
            dir_name += "/"

        upsert_folder(dir_name)

        with open(file_name, "w", encoding="utf-8") as f:
            f.write(str(content))

        return file_name

    def _update_crawler(self, soup, base_url, scrape_crawler):
        """if config is part of a scrape_crawler / managed threadpool will update crawler's urls_to_visit list."""

        scrape_crawler._add_url_to_visited(self.url)

        urls_to_visit = self.scrape_config.link_extractor_fn(soup, base_url)

        [scrape_crawler._add_url_to_visit(url) for url in urls_to_visit]

        return urls_to_visit

In [106]:
# |exporti


@patch_to(Scrape_Task)
def execute(
    self: Scrape_Task,
    base_url=None,
    driver=None,
    download_folder=None,
    debug_prn: bool = False,
    is_suppress_errors: bool = False,
    is_test: bool = False,
):
    """handles executing the scrape_task
    1. get pagesource
    2. download index
    3. download content
    4. update crawler
    - get_links from pagesource
    """
    url = self.url

    download_folder = download_folder or self.download_folder or "./SCRAPE"

    scrape_crawler = self.scrape_crawler

    driver = (
        driver
        or (scrape_crawler and scrape_crawler.driver_generator.get_webdriver())
        or None
    )

    if not driver:
        raise ScrapeTask_NoDriverProvided()

    base_url = (
        base_url
        or self.base_url
        or (scrape_crawler and scrape_crawler.base_url)
        or None
    )

    if debug_prn:
        print(f"scraping_page {url}")

    try:
        soup = self._get_pagesource(driver=driver, debug_prn=debug_prn)

        # download index
        file_name = self.generate_filename_fn(
            url=url, download_folder=download_folder, file_name="index.html"
        )

        save_location = self._download_content(file_name=file_name, content=soup)

        # download content
        if self.scrape_config:
            content = self.scrape_config.content_extractor_fn(soup)

            content_name = self.generate_filename_fn(
                url=url, download_folder=download_folder, file_name="content.html"
            )

            self._download_content(
                content_name,
                content=content,
            )

        print(f"🎉 successfully scraped {url} to {save_location}")

        if is_test:
            return []

        # update crawler
        if not base_url:
            raise ScrapeTask_NoBaseUrl()

        if not scrape_crawler:
            raise ScrapeTask_NoScrapeCrawler()

        """if config is part of a scrape_crawler / managed threadpool will update crawler's urls_to_visit list."""
        return self._update_crawler(
            soup=soup, base_url=base_url, scrape_crawler=scrape_crawler
        )

    except Exception as e:
        if not is_suppress_errors:
            raise (e)
        return f"💀 failed to download {url} received errror{e}"

In [107]:
task = Scrape_Task(
    url=test_urls[0],
)

str(task._get_pagesource(driver=driver))[0:100]

'<html dir="ltr" lang="en-US"><head><title>Article Detail</title><meta content="default-src \'self\'; s'

In [108]:
task = Scrape_Task(
    url=test_urls[2],
)

task.execute(driver=driver, base_url="https://domo-support.domo.com", is_test=True)

🎉 successfully scraped https://domo-support.domo.com/s/topic/0TO5w000000Zan7GAC/archived-feature-release-notes to ./SCRAPE/s_topic_0TO5w000000Zan7GAC_archived-feature-release-notes/index.html


[]

# Scrape Factory converts URL into Scrape_Config task

In [109]:
# | export


class Scrape_Factory_NoConfigMatch(Exception):
    def __init__(self, text):
        super().__init__(
            f"{text} has no pattern match in factory_configs, add an appropriate config or check pattern matches"
        )


@dataclass
class Scrape_Factory:
    """class handles a list of Scrape_Configs and will return the 'correct one' given a URL"""

    factory_configs: List[Scrape_Config]

    def get_task(
        self,
        url,
        download_folder="./SCRAPE",
        base_url=None,
        scrape_crawler=None,
        debug_prn: bool = False,
    ):
        config = next(
            (
                config
                for config in self.factory_configs
                if config.is_text_match_pattern(url, debug_prn=debug_prn)
            ),
            None,
        )

        if not config:
            raise Scrape_Factory_NoConfigMatch(text=url)

        return Scrape_Task._from_factory(
            scrape_config=config,
            url=url,
            base_url=base_url,
            download_folder=download_folder,
            scrape_crawler=scrape_crawler,
        )

## DomoKB_ScrapeFactory

An instance of Scrape Factory for scraping Domo KBs three types of pages, Articles, Topic lists, and the main page

In [110]:
# | exporti
def process_domo_kb_link(url, base_url):
    # remove query params
    url = remove_query_params_from_url(url)

    if not url or not "/s/" in url:
        return None

    # only keep the first 6 pieces of the URL
    url = "/".join(url.split("/")[:6])

    if url.endswith("/"):
        url = url[:-1]

    return url


def domokb_link_extractor_fn(soup, base_url, debug_prn: bool = False):
    """custom link extractor for processing Domo KBs.
    will be embedded into all DomoKB_ScrapeConfigs
    """
    return extract_links(
        soup,
        custom_link_extractor_fn=process_domo_kb_link,
        base_url=base_url,
        debug_prn=debug_prn,
    )

In [111]:
# | hide
domokb_link_extractor_fn(test_soup, "https://domo-support.domo.com")

['https://domo-support.domo.com/s/knowledge-base',
 'https://domo-support.domo.com/s/topic/0TO5w000000ZamwGAC']

## DomoKB_ScrapeConfig_Article

In [112]:
# | export


def domokb_article_content_extractor_fn(soup) -> BeautifulSoup:
    return soup.find(class_=["article-column"])


DomoKB_ScrapeConfig_Article = Scrape_Config(
    pattern=r".*/s/article/.*",
    link_extractor_fn=domokb_link_extractor_fn,
    content_extractor_fn=domokb_article_content_extractor_fn,
    search_element_type=By.CLASS_NAME,
    search_element_text="slds-form-element",
)

## DomoKB_ScrapeConfig_Topic

In [113]:
# | export
def domokb_topic_content_extractor_fn(soup) -> BeautifulSoup:
    return soup.find(class_=["knowledge-base"])


DomoKB_ScrapeConfig_Topic = Scrape_Config(
    pattern=r".*/s/topic/.*",
    link_extractor_fn=domokb_link_extractor_fn,
    content_extractor_fn=domokb_topic_content_extractor_fn,
    search_element_type=By.CSS_SELECTOR,
    search_element_text=f".{', .'.join(['section-list-item', 'article-list-item'] )}",
)

## DomoKB_ScrapeConfig_NavContainer

In [114]:
# |export
def domokb_knowledgebase_content_extractor_fn(soup) -> BeautifulSoup:
    return soup.find(class_=["knowledge-base"])


DomoKB_ScrapeConfig_KnowledgeBase = Scrape_Config(
    pattern=r".*/s/knowledge-base.*",
    link_extractor_fn=domokb_link_extractor_fn,
    content_extractor_fn=domokb_knowledgebase_content_extractor_fn,
    search_element_type=By.CSS_SELECTOR,
    search_element_text=f".{', .'.join(['topic-nav-container', 'cDomoKBCategoryNav'] )}",
)

## DomoKB_ScrapeFactory
A factory pattern receives a parameter (the url to match to) then returns the appropriate implementation `Scrape_Config` for handling that url.

This pattern could be extended for any type of website; however we have implemented a factory specifically for handling Domo Kbs

In [115]:
# | export
DomoKB_ScrapeFactory = Scrape_Factory(
    [
        DomoKB_ScrapeConfig_Article,
        DomoKB_ScrapeConfig_Topic,
        DomoKB_ScrapeConfig_KnowledgeBase,
    ]
)

#### sample implementation

Of retrieving the scrape_config using the `scrape_factory.get_factory_config() method

- test_urls is a list of URLS
- use a list comprenension to show we can retrieve the correct config for different URLs

In [116]:
test_scrape_tasks = [
    {
        "url": url,
        "config": DomoKB_ScrapeFactory.get_task(
            url, base_url="https://domo-support.domo.com", debug_prn=False
        ),
    }
    for url in test_urls
]

test_scrape_tasks

[{'url': 'https://domo-support.domo.com/s/article/36004740075',
  'config': Scrape_Task(url='https://domo-support.domo.com/s/article/36004740075', base_url='https://domo-support.domo.com', download_folder='./SCRAPE', generate_filename_fn=<function generate_filename_from_url at 0x7f96df933e20>, max_sleep_time=10, scrape_config=Scrape_Config(pattern='.*/s/article/.*', content_extractor_fn=<function domokb_article_content_extractor_fn at 0x7f96df7ba160>, link_extractor_fn=<function domokb_link_extractor_fn at 0x7f96df9320c0>, search_element_type='class name', search_element_text='slds-form-element'), scrape_crawler=None)},
 {'url': 'https://domo-support.domo.com/s/topic/0TO5w000000ZlOmGAK/20202023',
  'config': Scrape_Task(url='https://domo-support.domo.com/s/topic/0TO5w000000ZlOmGAK/20202023', base_url='https://domo-support.domo.com', download_folder='./SCRAPE', generate_filename_fn=<function generate_filename_from_url at 0x7f96df933e20>, max_sleep_time=10, scrape_config=Scrape_Config(pa

In [117]:
test_task = test_scrape_tasks[2]["config"]
test_task.__dict__

{'url': 'https://domo-support.domo.com/s/topic/0TO5w000000Zan7GAC/archived-feature-release-notes',
 'base_url': 'https://domo-support.domo.com',
 'download_folder': './SCRAPE',
 'generate_filename_fn': <function __main__.generate_filename_from_url(url, download_folder=None, file_name=None) -> 'str'>,
 'max_sleep_time': 10,
 'scrape_config': Scrape_Config(pattern='.*/s/topic/.*', content_extractor_fn=<function domokb_topic_content_extractor_fn at 0x7f96df7ba5c0>, link_extractor_fn=<function domokb_link_extractor_fn at 0x7f96df9320c0>, search_element_type='css selector', search_element_text='.section-list-item, .article-list-item'),
 'scrape_crawler': None}

In [118]:
driver_generator = dg.DriverGenerator()
driver = driver_generator.get_webdriver()

test_task.execute(driver=driver, is_test=True)

🎉 successfully scraped https://domo-support.domo.com/s/topic/0TO5w000000Zan7GAC/archived-feature-release-notes to ./SCRAPE/s_topic_0TO5w000000Zan7GAC_archived-feature-release-notes/index.html


[]

# Scrape_Crawler

The Scrape_Crawler is generally just a crawler that receieves a URL then manages scraping that page (by retrieving the correct scrape_configuration process) and searching for URLs to extract from that page and adding it to the `urls_to_visit` property.

The ThreadPoolExecutor has a maximum number of threads, maximum_workers, it will use.

This crawler does not concern itself with the return of the webcrawl task, because the task handles downloading the HTML file to a parameterized location 

In [119]:
# | export


@dataclass
class Scrape_Crawler:
    """threadpool manager for crawling through a list of urls"""

    executor: ThreadPoolExecutor
    scrape_factory: Scrape_Factory
    base_url: str

    download_folder: str
    visited_urls: set
    urls_to_visit: set

    driver_generator: dg.DriverGenerator = None

    def __init__(
        self,
        driver_path,
        scrape_factory: Scrape_Factory,
        urls_to_visit: list,
        base_url: str,
        urls_visited: list = None,
        max_workers=5,
        download_folder: str = "./SCRAPE/",
    ):
        self.base_url = base_url
        self.scrape_factory = scrape_factory
        self.executor = ThreadPoolExecutor(max_workers=max_workers)
        self.driver_generator = dg.DriverGenerator(driver_path=driver_path)
        self.download_folder = download_folder

        self.visited_urls = set()
        if urls_visited:
            [self._add_url_to_visited(url) for url in urls_visited]

        self.urls_to_visit = set()

        if urls_to_visit:
            [self._add_url_to_visit(url) for url in urls_to_visit]

    def _add_url_to_visit(self, url, debug_prn: bool = False):
        """adds a URL to the list of URLS to visit after testing that the URL has not already been visited"""

        if url not in self.visited_urls and url not in self.urls_to_visit:
            if debug_prn:
                print(f"adding {url} to to_vist list")

            self.urls_to_visit.add(url)
            return self.urls_to_visit

    def _add_url_to_visited(self, url, debug_prn: bool = False):
        if url not in self.visited_urls:
            if debug_prn:
                print(f"adding {url} to visited list")

            self.visited_urls.add(url)
            return self.visited_urls

    def _quit(self):
        """call when the executor queue is empty"""

        self.executor.shutdown(wait=True)
        return f"Done scraping {len(self.visited_urls)} urls"

In [120]:
wbs = Scrape_Crawler(
    driver_path="/usr//bin/chromedriver",
    scrape_factory=DomoKB_ScrapeFactory,
    base_url="https://domo-support.domo.com",
    urls_to_visit=test_urls,
)
wbs.__dict__

{'base_url': 'https://domo-support.domo.com',
 'scrape_factory': Scrape_Factory(factory_configs=[Scrape_Config(pattern='.*/s/article/.*', content_extractor_fn=<function domokb_article_content_extractor_fn at 0x7f96df7ba160>, link_extractor_fn=<function domokb_link_extractor_fn at 0x7f96df9320c0>, search_element_type='class name', search_element_text='slds-form-element'), Scrape_Config(pattern='.*/s/topic/.*', content_extractor_fn=<function domokb_topic_content_extractor_fn at 0x7f96df7ba5c0>, link_extractor_fn=<function domokb_link_extractor_fn at 0x7f96df9320c0>, search_element_type='css selector', search_element_text='.section-list-item, .article-list-item'), Scrape_Config(pattern='.*/s/knowledge-base.*', content_extractor_fn=<function domokb_knowledgebase_content_extractor_fn at 0x7f96df7ba8e0>, link_extractor_fn=<function domokb_link_extractor_fn at 0x7f96df9320c0>, search_element_type='css selector', search_element_text='.topic-nav-container, .cDomoKBCategoryNav')]),
 'executor': 

In [123]:
# | exporti


@patch_to(Scrape_Crawler)
def crawl_urls(self: Scrape_Crawler, is_test: bool = False, debug_prn: bool = False):
    while self.urls_to_visit:
        url = self.urls_to_visit.pop()
        self.visited_urls.add(url)

        driver = self.driver_generator.get_webdriver()

        task = self.scrape_factory.get_task(
            url, debug_prn=debug_prn, scrape_crawler=self
        )

        future = self.executor.submit(task.execute, driver=driver, is_test=is_test)
        try:
            future.result()

        except Exception as e:
            print(e)

    return self._quit()

In [124]:
wbs = Scrape_Crawler(
    driver_path="/usr//bin/chromedriver",
    scrape_factory=DomoKB_ScrapeFactory,
    base_url="https://domo-support.domo.com",
    urls_to_visit=test_urls,
)

wbs.crawl_urls()

wbs.__dict__

🎉 successfully scraped https://domo-support.domo.com/s/knowledge-base to ./SCRAPE/s_knowledge-base/index.html
['https://domo-support.domo.com/s/knowledge-base', 'https://domo-support.domo.com/s/topic/0TO5w000000ZamwGAC']
🎉 successfully scraped https://domo-support.domo.com/s/article/36004740075 to ./SCRAPE/s_article_36004740075/index.html
['https://domo-support.domo.com/s/article/36004740075', 'https://domo-support.domo.com/s/knowledge-base', 'https://domo-support.domo.com/s/article/7440921035671', 'https://domo-support.domo.com/s/article/360043630093', 'https://domo-support.domo.com/s/topic/0TO5w000000ZamzGAC', 'https://domo-support.domo.com/s/article/360043429693', 'https://domo-support.domo.com/s/article/360043429933', 'https://domo-support.domo.com/s/article/000005166', 'https://domo-support.domo.com/s/topic/0TO5w000000ZmOBGA0', 'https://domo-support.domo.com/s/topic/0TO5w000000ZamwGAC', 'https://domo-support.domo.com/s/article/360043931814', 'https://domo-support.domo.com/s/articl

KeyboardInterrupt: 

🎉 successfully scraped https://domo-support.domo.com/s/article/360042931774 to ./SCRAPE/s_article_360042931774/index.html
['https://domo-support.domo.com/s/article/36004740075', 'https://domo-support.domo.com/s/knowledge-base', 'https://domo-support.domo.com/s/topic/0TO5w000000ZammGAC', 'https://domo-support.domo.com/s/article/360042926054', 'https://domo-support.domo.com/s/article/360043630093', 'https://domo-support.domo.com/s/topic/0TO5w000000ZanLGAS', 'https://domo-support.domo.com/s/topic/0TO5w000000ZaojGAC', 'https://domo-support.domo.com/s/article/360042926274', 'https://domo-support.domo.com/s/article/360043429933', 'https://domo-support.domo.com/s/topic/0TO5w000000ZamwGAC', 'https://domo-support.domo.com/s/article/360043931814', 'https://domo-support.domo.com/s/article/360042931774', 'https://domo-support.domo.com/s/article/360043429953']


In [ ]:
# | hide
import nbdev

nbdev.nbdev_export()